In [ ]:
import cv2
import os
import numpy as np
import pickle
import mediapipe as mp
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
# Initialize MediaPipe Hand module
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# List of classes
classes = ["call", "dislike", "Fist", "four", "like", "one", "peace", "rock", "stop", "three", "two up"]

# Initialize empty lists for images and labels
image_files = []
labels = []

# Define target image size
target_size = (150, 150)

# Specify the full path to the 'Gesture sense' folder
base_folder = r"C:\Users\HP\Contacts\Desktop\Gesture sense"

In [ ]:
# Loop through each class folder
for class_name in classes:
    class_folder = os.path.join(base_folder, "Data", class_name)
    # Check if the class folder exists
    if not os.path.exists(class_folder):
        print(f"Class folder '{class_name}' does not exist.")
        continue
    # Loop through the files in the class folder
    for filename in os.listdir(class_folder):
        if filename.endswith(".jpg"):  # Check if the file has the '.jpg' extension
            filepath = os.path.join(class_folder, filename)
            print("Loading image:", filepath)  # Debug statement to check the loaded file path
            # Load image using OpenCV
            image = cv2.imread(filepath)
            if image is not None:  # Check if the image is loaded successfully
                print("Image loaded successfully")  # Debug statement
                # Resize the image to the target size
                image = cv2.resize(image, target_size)
                # Append image and label
                image_files.append(image)
                labels.append(class_name)
            else:
                print(f"Failed to load image: {filename}")
        else:
            print(f"Skipping non-image file: {filename}")

In [ ]:
# Convert lists to numpy arrays
data = np.array(image_files)
labels = np.array(labels)

In [ ]:
# Check if there are samples in the dataset
if len(data) == 0:
    print("No images loaded.")
else:
    # Now you have your resized image data and corresponding class labels ready for further processing
    print("Data shape:", data.shape)
    print("Labels shape:", labels.shape)

In [ ]:
 # Split data into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
 # Initialize empty lists for hand landmarks and labels
    hand_landmarks = []
    hand_labels = []

In [ ]:
 # Loop through the training data to detect hand landmarks using MediaPipe
    for image in x_train:
        # Convert image to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Detect hand landmarks
        results = hands.process(image_rgb)
        if results.multi_hand_landmarks:
            for hand_landmark in results.multi_hand_landmarks:
                # Extract hand landmarks and corresponding label
                landmark_list = [(lm.x, lm.y, lm.z) for lm in hand_landmark.landmark]  # Extract x, y, z coordinates
                hand_landmarks.append(landmark_list)
                hand_labels.append(y_train[len(hand_landmarks) - 1])  # Corresponding label

In [ ]:
# Convert hand landmarks to numpy array
    x_train_hand_landmarks = np.array(hand_landmarks)
    y_train_hand_labels = np.array(hand_labels)

In [ ]:
 # Flatten the hand landmarks
    x_train_flat = x_train_hand_landmarks.reshape(x_train_hand_landmarks.shape[0], -1)

In [ ]:
 # Initialize the Random Forest classifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)

In [ ]:
# Train the model
    model.fit(x_train_flat, y_train_hand_labels)

In [ ]:
 # Specify the directory path where you want to save the model
    model_directory = r"C:\Users\HP\Contacts\Desktop\Gesture sense\Model"

    # Specify the filename for the model
    model_filename = "hand_gesture_model_mediapipe.pkl"

    # Combine the directory path and filename
    model_filepath = os.path.join(model_directory, model_filename)

In [ ]:
# Save the trained model to the file
    with open(model_filepath, 'wb') as f:
        pickle.dump(model, f)

    print("Model saved as:", model_filepath)